# Import necessary libraries

In [26]:
!pip install finnhub-python
import yfinance as yf
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta, date
import json


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


# Import Data

In [ ]:
import os
current_directory = os.getcwd()  # Get the current working directory
print(current_directory)  # Outputs the current directory
subdirectory = 'manual_inputs'  # This is your target subfolder
file_name = 'all_tickers.txt'  # Name of the file to save
file_path = os.path.join(current_directory, subdirectory, file_name)  # Full path for the file

with open(file_path, 'r') as file:
    all_tickers = file.read().splitlines()

# with open('all_tickers.txt', 'r') as file:
#     all_tickers = file.read().splitlines()
    
# Market Indices
tickers = [
    '^IXIC',
    '^VIX',  # CBOE Volatility Index
    '^TNX',  # 10-Year Treasury Note Yield
    '^IRX',  # 13-Week Treasury Bill Yield
]

# Bonds and Fixed Income ETFs
tickers += [
    'TLT',   # iShares 20+ Year Treasury Bond ETF
    'TMF',   # Direxion Daily 20-Year Treasury Bull 3X Shares
    'SHY',   # iShares 1-3 Year Treasury Bond ETF
    'EDV',   # Vanguard Extended Duration Treasury ETF
    'ZROZ',  # PIMCO 25+ Year Zero Coupon U.S. Treasury Index ETF
    'GOVT',  # Vanguard U.S. Government Bond ETF
]

# Volatility and Inverse ETFs
tickers += [
    'VXX',   # iPath Series B S&P 500 VIX Short-Term Futures ETN
    'UVXY',  # ProShares Ultra VIX Short-Term Futures ETF
]

# Growth and Value ETFs
tickers += [
    'MTUM',  # iShares Edge MSCI USA Momentum Factor ETF
    'SPY',   # SPDR S&P 500 ETF Trust
    'QQQ',   # Invesco QQQ Trust (Nasdaq 100)
    'QQQM',  # Invesco NASDAQ 100 ETF (mini)
    'VOOG',  # Vanguard S&P 500 Growth ETF
    'VGT',   # Vanguard Information Technology ETF
    'XLU',   # Utilities Select Sector SPDR Fund
]

# Large-Cap Stocks
tickers += [
    'GOOG',  # Alphabet Inc. (Class C)
    'AVGO',  # Broadcom Inc.
    'AMZN',  # Amazon.com, Inc.
    'NVDA',  # NVIDIA Corporation
    'MSFT',  # Microsoft Corporation
    'DIS',   # The Walt Disney Company
    'AAPL',  # Apple Inc.
    'WMT',   # Walmart Inc.
    'META',  # Meta Platforms, Inc. (formerly Facebook)
    'COST',  # Costco Wholesale Corporation
]

# Precious Metals
tickers += [
    'GLD',   # SPDR Gold Shares
    'IAU',   # iShares Gold Trust
    'PHYS',  # Sprott Physical Gold Trust
    'GDX',   # VanEck Vectors Gold Miners ETF
    'SLV',   # iShares Silver Trust
    'IAU',   # iShares Gold Trust (duplicate)
]

# Small and Mid-Cap Stocks / Growth Companies
tickers += [
    'SFM',   # Sprouts Farmers Market, Inc.
    'PLTR',  # Palantir Technologies Inc.
    'QUBT',  # Quantum Computing Inc.
    'IONQ',  # IonQ, Inc. (quantum computing)
    'APLD',  # Applied Digital Corporation
    'LMND',  # Lemonade, Inc.
    'VRT',   # Vertiv Holdings Co.
    'RKLB',  # Rocket Lab USA, Inc.
    'AFRM',  # Affirm Holdings, Inc.
    'FLR',   # Fluor Corporation
    'ESOA',  # Eos Energy Enterprises, Inc.
    'BKNG',  # Booking Holdings Inc.
    'HQY',   # HealthEquity, Inc.
    'NU',    # Nu Holdings Ltd.
    'VNO',   # Vornado Realty Trust
    'MP',    # MP Materials Corp.
    'SMR',   # SM Energy Company
    'IDCC',  # InterDigital, Inc.
    'RVMD',  # Reviva Pharmaceuticals Holdings, Inc.
    'DHI',   # D.R. Horton, Inc.
    'PANW',  # Palo Alto Networks, Inc.
    'DASH',  # DoorDash, Inc.
    'CHWY',  # Chewy, Inc.
    'WPM',   # Wheaton Precious Metals Corp.
    'SE',    # Sea Limited
     'T',    # AT&T Inc.
    'LMT',   # Lockheed Martin Corporation
    'TKO',   # Take-Two Interactive Software, Inc.
    'LNTH',  # Lantheus Holdings, Inc.
    'FICO',  # Fair Isaac Corporation (FICO)
]

# Chemicals and Energy
tickers += [
    'CL',    # Continental Resources, Inc.
    'NEE',   # NextEra Energy, Inc.
    'CCJ',   # Cameco Corporation
    'GEVO',  # Gevo, Inc.
]

# Check which tickers in your tickers list are not in all_tickers
missing_tickers = [ticker for ticker in tickers if ticker not in all_tickers]

# Print results
if not missing_tickers:
    print("All tickers are included in all_tickers.")
else:
    print("Missing tickers:", missing_tickers)
    
all_tickers += missing_tickers
all_tickers.sort()

# tickers = all_tickers

Missing tickers: ['^IXIC', '^VIX', '^TNX', '^IRX', 'TLT', 'TMF', 'SHY', 'EDV', 'ZROZ', 'GOVT', 'VXX', 'UVXY', 'MTUM', 'SPY', 'QQQ', 'QQQM', 'VOOG', 'VGT', 'XLU', 'GLD', 'IAU', 'PHYS', 'GDX', 'SLV', 'IAU']


# Technical Indicators

## Custom Indicators - High Momentum Factor / High Quality Score (HMHQ)

In [7]:
%%capture

# Initialize a dictionary to store results for each ticker
results = {}

# Define a target return (for instance, the risk-free rate)
target_return = 0.0  # Adjust this as needed
rolling_window = 20   # Define the window size for rolling calculations

# Define the benchmark
benchmark_ticker = 'SPY'

# ** Data Processing Loop for Each Ticker **
for ticker in tickers:
    # ** 0A. Data Downloading Section **
    try:
        data = yf.download(ticker, start=(datetime.now() - timedelta(days=2*365)).strftime("%Y-%m-%d"), end=datetime.now().strftime("%Y-%m-%d"), interval="1d")

        # Check if the data is empty
        if data.empty:
            print(f"No data found for {ticker}. Skipping.")
            continue
        
        # Add benchmark data for Beta calculation
        if ticker == benchmark_ticker:
            benchmark_data = data['Close']
            continue  # Skip the rest for the benchmark itself

        # ** 0. Moving Averages Section **
        data['3_SMA'] = data['Close'].rolling(window=3).mean()
        data['5_SMA'] = data['Close'].rolling(window=5).mean()
        data['20_SMA'] = data['Close'].rolling(window=20).mean()
        data['50_SMA'] = data['Close'].rolling(window=50).mean()
        data['90_SMA'] = data['Close'].rolling(window=90).mean()
        data['180_SMA'] = data['Close'].rolling(window=180).mean()
        data['360_SMA'] = data['Close'].rolling(window=360).mean()
   
        # ** 0. Price Change Calculations Section **
        data['Price_Change%'] = data['Close'].pct_change() * 100 
        data['Cumulative_Returns'] = (1 + data['Price_Change%'] / 100).cumprod()
             
        # ** 1. Market Trend Section - when 3_SMA > 5_SMA **
        data['Market_Trend'] = 0
        data.loc[data['3_SMA'] > data['5_SMA'], 'Market_Trend'] = 1
        data.loc[data['3_SMA'] < data['5_SMA'], 'Market_Trend'] = -1

        # ** 2. Rolling Maximum Drawdown Calculation Section **
        data['Rolling_Peak'] = data['Cumulative_Returns'].cummax()
        data['Rolling_Drawdown'] = (data['Cumulative_Returns'] - data['Rolling_Peak']) / data['Rolling_Peak']

        # ** 3. Cumulative Change Calculation Section using vectorized operations **
        current_trend_shifted = data['Market_Trend'].shift(1)
        current_base_price = data['Close'].shift(1).where(current_trend_shifted != data['Market_Trend']).ffill()
        data['Cumulative_Change%'] = ((data['Close'] - current_base_price) / current_base_price) * 100
        data['Over_10%_in_90_days'] = data['Cumulative_Change%'].rolling(window=90, min_periods=1).max() > 10
        data['Over_10%_in_90_days'] = data['Over_10%_in_90_days'].astype(int)

        # ** 4. Rolling Up/Down Ratio Calculation **
        data['Up_Day'] = data['Price_Change%'].apply(lambda x: x if x > 0 else 0)
        data['Down_Day'] = data['Price_Change%'].apply(lambda x: -x if x < 0 else 0)
        data['Rolling_Up_Down_Ratio'] = data['Up_Day'].rolling(window=rolling_window).mean() / data['Down_Day'].rolling(window=rolling_window).mean()
        
        # ** 5. Rolling True Strength Index (TSI) Calculation **
        data['Smooth_Price_Change'] = data['Price_Change%'].ewm(span=25, adjust=False).mean()  # Fast TSI
        data['Smooth_Price_Change_Slow'] = data['Price_Change%'].ewm(span=13, adjust=False).mean()  # Slow TSI
        data['TSI'] = 100 * (data['Smooth_Price_Change'] - data['Smooth_Price_Change_Slow']) / data['Smooth_Price_Change_Slow']
        data['Rolling_TSI'] = data['TSI'].rolling(window=rolling_window).mean()

        # ** 6. Rolling Volatility Calculation **
        data['Rolling_Volatility'] = data['Price_Change%'].rolling(window=rolling_window).std()

        # ** 7. Rolling Maximum Consecutive Up Days Calculation **
        data['Consecutive_Up_Days'] = (data['Price_Change%'] > 0).astype(int)
        data['Rolling_Consecutive_Up_Days'] = data['Consecutive_Up_Days'].groupby((data['Consecutive_Up_Days'] != data['Consecutive_Up_Days'].shift()).cumsum()).cumsum()
        
        # ** 8. Max Rolling Consecutive Up Days in a 20-day Window **
        consecutive_counts = data['Consecutive_Up_Days'].rolling(window=rolling_window).apply(
            lambda x: (x != 0).astype(int).groupby((x != 0).ne((x.shift())).cumsum()).cumsum().max(),
            raw=False
        )
        data['Max_Rolling_Consecutive_Up_Days'] = consecutive_counts
        
        consecutive_counts = data['Consecutive_Up_Days'].rolling(window=90).apply(
            lambda x: (x != 0).astype(int).groupby((x != 0).ne((x.shift())).cumsum()).cumsum().max(),
            raw=False
        )
        data['Max_SemiAnnual_Consecutive_Up_Days'] = consecutive_counts
        data['Potential_Up_Days'] = data['Max_SemiAnnual_Consecutive_Up_Days']-data['Consecutive_Up_Days']

        # ** 9. RSI Calculation (Relative Strength Index) **
        delta = data['Close'].diff(1)
        gain = (delta.where(delta > 0, 0)).rolling(window=rolling_window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=rolling_window).mean()
        rs = gain / loss
        data['RSI'] = 100 - (100 / (1 + rs))

        # ** 10. Momentum Calculation **
        data['Momentum'] = data['Close'].diff(rolling_window)

        # ** 11. ATR Calculation (Average True Range) **
        data['High_Low'] = data['High'] - data['Low']
        data['High_Close'] = (data['High'] - data['Close'].shift(1)).abs()
        data['Low_Close'] = (data['Low'] - data['Close'].shift(1)).abs()
        data['True_Range'] = data[['High_Low', 'High_Close', 'Low_Close']].max(axis=1)
        data['ATR'] = data['True_Range'].rolling(window=rolling_window).mean()

        # ** 12. Beta Calculation **
        if 'benchmark_data' in locals():
            returns = data['Close'].pct_change()
            benchmark_returns = benchmark_data.pct_change()
            covariance = returns.rolling(window=rolling_window).cov(benchmark_returns)
            variance = benchmark_returns.rolling(window=rolling_window).var()

            data['Beta'] = covariance / variance

        # ** 13. Alpha Calculation **
        if 'benchmark_data' in locals():
            average_benchmark_return = benchmark_data.pct_change().rolling(window=rolling_window).mean()
            data['Alpha'] = (data['Price_Change%'].rolling(window=rolling_window).mean() - (data['Beta'] * average_benchmark_return)).fillna(0)

        # ** 14. Preparing the Trend Indicator DataFrame **
        trend_indicator = data[['Close', 
                                'Price_Change%', 
                                'Market_Trend', 
                                'Cumulative_Change%', 
                                'Over_10%_in_90_days',
                                'Rolling_Drawdown', 
                                'Rolling_Up_Down_Ratio', 
                                'Rolling_TSI', 
                                'Rolling_Volatility', 
                                'Rolling_Consecutive_Up_Days', 
                                'Max_Rolling_Consecutive_Up_Days', 
                                'Max_SemiAnnual_Consecutive_Up_Days',
                                'Potential_Up_Days',
                                'RSI', 
                                'Momentum',
                                'ATR']].dropna()

        # Conditionally adding Alpha and Beta if they exist in the DataFrame
        trend_columns = [
            f'{ticker} Close',
            f'{ticker} daily move%',
            f'{ticker} ST trend signal',
            f'{ticker} cumulative change since signal',
            f'{ticker} cumulative change 90-day breakthrough',
            f'{ticker} Rolling Max Drawdown',
            f'{ticker} Rolling Up Down Ratio',
            f'{ticker} Rolling TSI',
            f'{ticker} Rolling Volatility',
            f'{ticker} Consecutive Up Days',
            f'{ticker} Max Rolling Consecutive Up Days',
            f'{ticker} Max_SemiAnnual_Consecutive_Up_Days',
            f'{ticker} Potential_Up_Days',
            f'{ticker} RSI',
            f'{ticker} Momentum',
            f'{ticker} ATR'
        ]

        # Check if Alpha and Beta exist in data for column addition
        if 'Alpha' in data.columns:
            trend_indicator[f'{ticker} Alpha'] = data['Alpha']
            trend_columns.append(f'{ticker} Alpha')

        if 'Beta' in data.columns:
            trend_indicator[f'{ticker} Beta'] = data['Beta']
            trend_columns.append(f'{ticker} Beta')

        trend_indicator.columns = trend_columns
        
        # ** 15. Sortino Ratio Calculation Section **
        average_return = data['Price_Change%'].rolling(window=rolling_window).mean()
        downside_returns = data['Price_Change%'][data['Price_Change%'] < target_return].rolling(window=rolling_window)
        downside_deviation = downside_returns.apply(lambda x: (x ** 2).mean() ** 0.5 if not x.empty else 0)
        data['Sortino Ratio'] = (average_return - target_return) / downside_deviation
        data['Sortino Ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)  # Replace infinities

        # ** 16. Calmar Ratio Calculation Section **
        annualized_return = ((1 + average_return / 100) ** 252) - 1
        data['Calmar Ratio'] = annualized_return / abs(data['Rolling_Drawdown'].min())  # Use current rolling drawdown

        # ** 17. Sharpe Ratio Calculation Section **
        trend_indicator[f'{ticker} Sortino Ratio'] = data['Sortino Ratio']
        trend_indicator[f'{ticker} Calmar Ratio'] = data['Calmar Ratio']

        results[ticker] = trend_indicator  # Store the results for this ticker

    except Exception as e:
        print(f"Error processing {ticker}: {e}")

# ** Risk-Free Rate Calculation Section **
if "^TNX" in results:  # Ensure ^TNX has been processed successfully
    treasury_yield = results["^TNX"]['^TNX Close']
    risk_free_rate = treasury_yield / 100 / 252  # Convert from annual to daily
else:
    print("Warning: ^TNX results not found. Defaulting risk-free rate to 0.")
    risk_free_rate = 0.0  # Fallback risk-free rate

# ** Calculate Sharpe Ratio for all tickers using the risk-free rate **
for ticker in tickers:
    if ticker in results:
        data = results[ticker]
        rolling_average_return = data[f'{ticker} daily move%'].rolling(window=rolling_window).mean()
        rolling_std_dev_returns = data[f'{ticker} daily move%'].rolling(window=rolling_window).std()

        # Define Sharpe Ratio; replace infinities for stability
        data[f'{ticker} Sharpe Ratio'] = (rolling_average_return - risk_free_rate) / rolling_std_dev_returns
        data[f'{ticker} Sharpe Ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)  # Replace infinities
        results[ticker] = data  # Update results with Sharpe Ratio

# ** Display Results Section **
# for ticker in tickers:
#     if ticker in results:  # Only display results for tickers that were processed successfully
#         print(f"Results for {ticker}:")
#         display(results[ticker].tail(2))  # Display the last 10 entries

# Get the indices where the daily move percentage is less than -2%
high_move_indices = results['^IXIC'][results['^IXIC']['^IXIC daily move%'] < -2].index

# Convert DatetimeIndex to a list of datetime objects
date_list = high_move_indices.tolist()

# Convert the dates to strings formatted as 'YYYY-MM-DD'
date_list_as_strings = [date.strftime('%Y-%m-%d') for date in date_list]

# Display the formatted list of dates
print("List of dates with a daily move less than -2%:", date_list_as_strings)

# Iterate over each ticker in the results
for ticker in results:
    # Access the DataFrame for the current ticker
    df = results[ticker]
    
    # Create a new column for the reversed_trend_flag, initially set to 0
    df['anti-drawdown flag'] = 0
    
    # Convert date_list_as_strings to datetime objects for comparison
    date_list_as_datetimes = pd.to_datetime(date_list_as_strings)
    
    # Create a mask that checks for dates in date_list_as_datetimes
    mask = df.index.isin(date_list_as_datetimes)
    
    # Set the reversed_trend_flag to 1 for those dates if daily move% > -0.5%
    df.loc[mask, 'anti-drawdown flag'] = (df.loc[mask, f'{ticker} daily move%'] > -0.5).astype(int)

    # Update the results dictionary with the modified DataFrame
    results[ticker] = df
    
    # mask = results['SFM'].index.isin(date_list_as_datetimes)
    # filtered_df = results['SFM'][mask]
    # filtered_df
    
    # Set your rolling window size
    rolling_window_size = 180  # Adjust this as needed

    # Calculate the rolling average of 'anti-drawdown flag' and create a new column
    results[ticker]['Rolling Average Anti Drawdown'] = results[ticker]['anti-drawdown flag'].rolling(window=rolling_window_size).sum()


In [23]:
import pandas as pd

# Create a Pandas Excel writer using XlsxWriter as the engine.
with pd.ExcelWriter('technical_indicators_HMHQ.xlsx', engine='xlsxwriter') as writer:
    for ticker, df in results.items():
        df.to_excel(writer, sheet_name=ticker)  # Save each ticker's DataFrame to a separate sheet

In [ ]:
# Initialize an empty list to store modified DataFrames
modified_dfs = []

# Iterate over all tickers and their corresponding DataFrames
for ticker, df in results.items():
    # Reset the index to make the date a column
    df_reset = df.reset_index(drop=False)  # drop=False keeps the index

    # Add a new column for the ticker
    df_reset['Ticker'] = ticker

    # Rename columns to remove the ticker string
    # Assuming columns with ticker are structured like '{ticker} ColumnName'
    df_reset.columns = [col.replace(f'{ticker} ', '') for col in df_reset.columns]
    
    # Drop the 'Trend signal' column if it exists    
    if 'Strend signal' in df_reset.columns:        
        df_reset.drop(columns=['Strend signal'], inplace=True)
    
    # Append the modified DataFrame to the list
    modified_dfs.append(df_reset)

# Combine all modified DataFrames into a single DataFrame
combined_df = pd.concat(modified_dfs, ignore_index=True)

# Now you can save the combined DataFrame to a CSV file
combined_df.to_csv('technical_indicators_HMHQ.csv', index=False)

# Print out the first few rows of the combined DataFrame
display(combined_df.head())


,Date,Close,daily move%,ST trend signal,cumulative change since signal,cumulative change 90-day breakthrough,Rolling Max Drawdown,Rolling Up Down Ratio,Rolling TSI,Rolling Volatility,...,Momentum,ATR,Alpha,Beta,Sortino Ratio,Calmar Ratio,Sharpe Ratio,anti-drawdown flag,Rolling Average Anti Drawdown,Ticker
0,2023-03-21,11860.110352,1.580829,1.0,3.779789,0,-0.027925,1.395715,-67.325230,1.282598,...,367.810547,215.817432,0.165402,1.079563,NaN,3.930468,NaN,0,NaN,^IXIC
1,2023-03-22,11669.959961,-1.603277,1.0,2.115912,0,-0.043510,1.158259,-63.916822,1.342291,...,162.889648,226.306982,0.079654,1.075255,0.058187,1.671294,NaN,0,NaN,^IXIC
2,2023-03-23,11787.400391,1.006348,1.0,3.143553,0,-0.033885,1.186577,-59.565780,1.350886,...,197.000000,230.623486,0.093911,1.082886,NaN,2.006795,NaN,0,NaN,^IXIC
3,2023-03-24,11823.959961,0.310158,-1.0,0.310158,0,-0.030888,1.465754,-57.465440,1.284620,...,429.019531,225.604980,0.192804,1.038756,NaN,4.752010,NaN,0,NaN,^IXIC
4,2023-03-27,11768.839844,-0.466173,1.0,-0.466173,0,-0.035406,1.315321,-54.294588,1.288319,...,301.859375,225.298486,0.137964,1.031187,0.101535,3.157224,NaN,0,NaN,^IXIC


In [9]:
%%capture

# Initialize a dictionary to hold the correlations with target stock
correlation_results = {}

# List of relevant indicators for correlation analysis
indicators = [
    'ST trend signal',
    'cumulative change since signal',
    'cumulative change 90-day breakthrough',
    'Rolling Max Drawdown',
    'Rolling Average Anti Drawdown',
    'Rolling Up Down Ratio',
    'Rolling TSI',
    'Rolling Volatility',
    'Potential_Up_Days',
    'Consecutive Up Days',
    'Max Rolling Consecutive Up Days',
    'RSI',
    'Momentum',
    'ATR',
    'Alpha',
    'Beta',
    'Sortino Ratio',
    'Calmar Ratio',
    'Sharpe Ratio'
]

# Create a list of DataFrames for other tickers
other_tickers_dfs = results

# Calculate correlations for each indicator with AAPL
for ticker, df_other in other_tickers_dfs.items():
    correlations = {}
    for indicator in indicators:
        focus_col = f'SFM {indicator}'
        other_col = f'{ticker} {indicator}'

        # Ensure both columns exist before calculating
        if focus_col in other_tickers_dfs['SFM'].columns and other_col in df_other.columns:
            try:
                correlation = other_tickers_dfs['SFM'][focus_col].corr(df_other[other_col])
                correlations[indicator] = correlation
            except Exception as e:
                print(f"Error calculating correlation for {ticker}: {str(e)}")

    # Calculate the ensemble correlation for the current ticker
    if correlations:  # Check if any correlations were calculated
        ensemble_correlation = np.mean(list(correlations.values()))
        correlation_results[ticker] = ensemble_correlation

# Convert the results dictionary into a DataFrame for better readability
results_df = pd.DataFrame(correlation_results.items(), columns=['Ticker', 'Ensemble Correlation'])

# Sort results_df by 'Ensemble Correlation' in descending order
sorted_results_df = results_df.sort_values(by='Ensemble Correlation', ascending=False)

In [14]:
# Initialize a dictionary to hold the correlations with the target stock
correlation_results = {}

# List of relevant indicators for correlation analysis
indicators = [
    'ST trend signal',
    'cumulative change since signal',
    'cumulative change 90-day breakthrough',
    'Rolling Max Drawdown',
    'Rolling Average Anti Drawdown',
    'Rolling Up Down Ratio',
    'Rolling TSI',
    'Rolling Volatility',
    'Potential_Up_Days',
    'Consecutive Up Days',
    'Max Rolling Consecutive Up Days',
    'RSI',
    'Momentum',
    'ATR',
    'Alpha',
    'Beta',
    'Sortino Ratio',
    'Calmar Ratio',
    'Sharpe Ratio'
]

# Define relative weights for each indicator
weights = [
    5,    # Weight for 'ST trend signal'
    15,   # Weight for 'cumulative change since signal'
    15,    # Weight for 'cumulative change 90-day breakthrough'
    15,   # Weight for 'Rolling Max Drawdown'
    15,   # Weight for 'Rolling Average Anti Drawdown'
    20,   # Weight for 'Rolling Up Down Ratio'
    10,   # Weight for 'Rolling TSI'
    5,    # Weight for 'Rolling Volatility'
    15,   # Weight for 'Potential_Up_Days'
    5,    # Weight for 'Consecutive Up Days'
    10,   # Weight for 'Max Rolling Consecutive Up Days'
    20,   # Weight for 'RSI'
    15,   # Weight for 'Momentum'
    5,    # Weight for 'ATR'
    5,    # Weight for 'Alpha'
    5,    # Weight for 'Beta'
    15,   # Weight for 'Sortino Ratio'
    15,   # Weight for 'Calmar Ratio'
    15    # Weight for 'Sharpe Ratio'
]

# Create a list of DataFrames for other tickers
other_tickers_dfs = results

# Calculate correlations for each indicator with SFM
for ticker, df_other in other_tickers_dfs.items():
    correlations = {}
    for indicator in indicators:
        focus_col = f'SFM {indicator}'
        other_col = f'{ticker} {indicator}'

        # Ensure both columns exist before calculating
        if focus_col in other_tickers_dfs['SFM'].columns and other_col in df_other.columns:
            try:
                correlation = other_tickers_dfs['SFM'][focus_col].corr(df_other[other_col])
                correlations[indicator] = correlation
            except Exception as e:
                print(f"Error calculating correlation for {ticker} using {indicator}: {str(e)}")

    # Calculate the weighted ensemble correlation for the current ticker
    if correlations:  # Check if any correlations were calculated
        weighted_sum = sum(correlations[indicator] * weights[i] for i, indicator in enumerate(indicators) if indicator in correlations)
        correlation_results[ticker] = {"weighted_sum": weighted_sum, "correlations": correlations}
    else:
        print(f"No valid correlations calculated for {ticker}")

# Debug: Print correlation_results to check for NaN values
# print("Correlation Results:", correlation_results)

# After all results are calculated, determine the min and max of the weighted sums
if correlation_results:
    valid_sums = [v["weighted_sum"] for v in correlation_results.values() if pd.notna(v["weighted_sum"])]

    min_weighted_sum = min(valid_sums) if valid_sums else 0
    max_weighted_sum = max(valid_sums) if valid_sums else 100  # To avoid max equal to min

    # Normalizing the weighted sums to a range of 0 to 100
    normalized_results = {}
    simple_averages = {}

    for ticker, result in correlation_results.items():
        weighted_sum = result["weighted_sum"]
        correlations = result["correlations"]

        # Calculate simple average only of valid correlations
        valid_correlations = [v for v in correlations.values() if pd.notna(v)]
        simple_average = sum(valid_correlations) / len(valid_correlations) if valid_correlations else 0
        
        # Normalize the weighted sum
        normalized_score = ((weighted_sum - min_weighted_sum) / (max_weighted_sum - min_weighted_sum) * 100) if max_weighted_sum > min_weighted_sum else 0

        normalized_results[ticker] = normalized_score
        simple_averages[ticker] = simple_average

# Convert the results into a DataFrame
results_df = pd.DataFrame({
    'Reporting_Date': datetime.now().strftime("%Y-%m-%d"),
    'Ticker': normalized_results.keys(),
    'Normalized Weighted Score': normalized_results.values(),
    'Simple Average Score': simple_averages.values()
})

# Sort results_df by 'Normalized Weighted Score' in descending order
sorted_normalized_results_df = results_df.sort_values(by='Normalized Weighted Score', ascending=False)
sorted_normalized_results_df['Ranking'] = sorted_normalized_results_df['Normalized Weighted Score'].rank(method='min', ascending=False)

# Display the sorted results
print("Sorted Normalized Weighted Scores with Simple Averages:")
display(sorted_normalized_results_df[:5])

sorted_normalized_results_df.to_csv('HMHQ_score_ranking.csv', index=False)

Sorted Normalized Weighted Scores with Simple Averages:


/opt/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


,Reporting_Date,Ticker,Normalized Weighted Score,Simple Average Score,Ranking
33,2024-11-08,SFM,100.000000,1.000000,1.0
49,2024-11-08,SMR,34.520711,0.207802,2.0
50,2024-11-08,IDCC,31.580485,0.215100,3.0
30,2024-11-08,PHYS,30.689850,0.179418,4.0
63,2024-11-08,CL,29.352112,0.159810,5.0


In [175]:
import certifi
import json

def get_jsonparsed_data(url):
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)

url = (f"https://financialmodelingprep.com/api/v3/etf/list?apikey={api_key}")
print(get_jsonparsed_data(url))

/var/folders/mq/yl1c66r91wb1n1hmpbygjj040000gn/T/ipykernel_12504/3835437075.py:5: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


[{'symbol': 'MEDX', 'name': 'Horizon Kinetics Medical ETF', 'price': 29.0653, 'exchange': 'NASDAQ Global Market', 'exchangeShortName': 'NASDAQ', 'type': 'etf'}, {'symbol': 'MZZ', 'name': 'ProShares UltraShort MidCap400', 'price': 8.9122, 'exchange': 'New York Stock Exchange Arca', 'exchangeShortName': 'AMEX', 'type': 'etf'}, {'symbol': 'NUAG', 'name': 'Nuveen Enhanced Yield U.S. Aggregate Bond ETF', 'price': 20.87, 'exchange': 'New York Stock Exchange Arca', 'exchangeShortName': 'AMEX', 'type': 'etf'}, {'symbol': 'IHYU.MI', 'name': 'iShares $ High Yield Corp Bond UCITS ETF USD (Dist)', 'price': 89.9, 'exchange': 'Milan', 'exchangeShortName': 'MIL', 'type': 'etf'}, {'symbol': 'PJFM', 'name': 'PGIM ETF Trust - PGIM Jennison Focused Mid-Cap ETF', 'price': 58.1125, 'exchange': 'New York Stock Exchange Arca', 'exchangeShortName': 'AMEX', 'type': 'etf'}, {'symbol': 'BGRO', 'name': 'BlackRock Large Cap Growth ETF', 'price': 33.3069, 'exchange': 'Nasdaq', 'exchangeShortName': 'NASDAQ', 'type':

In [ ]:
url_etf_stock_exposure = requests.get(url_etf_stock_exposure)

In [141]:
url_spy_holder = f"https://financialmodelingprep.com/api/v3/etf-holder/SPY?apikey={api_key}"
spy_holder = requests.get(url_spy_holder)
spy_holder.raise_for_status()  # Raises an error for bad responses
spy_holder = spy_holder.json()  # Parse JSON response
df_spy_holder = pd.DataFrame(spy_holder)
df_spy_holder = df_spy_holder.sort_values(by='weightPercentage', ascending=False)
df_spy_holder = df_spy_holder[['asset', 'weightPercentage']]
df_spy_holder.rename(columns={'asset': 'ticker', 'weightPercentage':'SPYweightPercentage'}, inplace=True)
df_spy_holder

,ticker,SPYweightPercentage
0,NVDA,7.252
1,AAPL,6.849
2,MSFT,6.262
3,AMZN,3.886
4,META,2.560
...,...,...
499,WBA,0.013
500,FOX,0.011
501,AMTM,0.008
502,NWS,0.007


In [185]:

url_etf_stock_exposure = f"https://financialmodelingprep.com/api/v3/etf-stock-exposure/{symbol}?apikey={api_key}"
etf_stock_exposure = requests.get(url_etf_stock_exposure)
etf_stock_exposure.raise_for_status()  # Raises an error for bad responses
etf_stock_exposure = etf_stock_exposure.json()  # Parse JSON response
df_etf_stock_exposure = pd.DataFrame(etf_stock_exposure)
df_etf_stock_exposure
df_etf_stock_exposure = df_etf_stock_exposure.sort_values(by='weightPercentage', ascending=False)
df_etf_stock_exposure = df_etf_stock_exposure[['etfSymbol', 'weightPercentage', 'assetExposure']]
df_etf_stock_exposure.rename(columns={'etfSymbol': 'ETF', 'assetExposure': 'ticker', 'weightPercentage': 'ETF Weightage'}, inplace=True)
df_etf_stock_exposure


,ETF,ETF Weightage,ticker
355,IUCS.L,13.053940,WMT
633,2B7D.DE,12.307420,WMT
470,FSTA,11.000783,WMT
78,VDC,10.860000,WMT
11,XLP,9.109784,WMT
...,...,...,...
454,GPAL,0.000000,WMT
455,GLOV,0.000000,WMT
458,GINN,0.000000,WMT
309,JUST,0.000000,WMT


In [180]:
df_etf_stock_exposure

,ETF,ETF Weightage,Ticker
0,LEND,0.09,AIHS


In [191]:
tickers

['^IXIC',
 '^VIX',
 '^TNX',
 '^IRX',
 'TLT',
 'TMF',
 'SHY',
 'EDV',
 'ZROZ',
 'GOVT',
 'VXX',
 'UVXY',
 'MTUM',
 'SPY',
 'QQQ',
 'QQQM',
 'VOOG',
 'VGT',
 'XLU',
 'GOOG',
 'AVGO',
 'AMZN',
 'NVDA',
 'MSFT',
 'DIS',
 'AAPL',
 'WMT',
 'META',
 'COST',
 'GLD',
 'IAU',
 'PHYS',
 'GDX',
 'SLV',
 'IAU',
 'SFM',
 'PLTR',
 'QUBT',
 'IONQ',
 'APLD',
 'LMND',
 'VRT',
 'RKLB',
 'AFRM',
 'FLR',
 'ESOA',
 'BKNG',
 'HQY',
 'NU',
 'VNO',
 'MP',
 'SMR',
 'IDCC',
 'RVMD',
 'DHI',
 'PANW',
 'DASH',
 'CHWY',
 'WPM',
 'SE',
 'T',
 'LMT',
 'TKO',
 'LNTH',
 'FICO',
 'CL',
 'NEE',
 'CCJ',
 'GEVO']

In [192]:
%%capture

# Initialize an empty list to store DataFrames
df_list = []

for symbol in tickers:
    # Construct the URL for the API request
    url_etf_stock_exposure = f"https://financialmodelingprep.com/api/v3/etf-stock-exposure/{symbol}?apikey={api_key}"
    
    try:
        # Make the API request
        response = requests.get(url_etf_stock_exposure)
        response.raise_for_status()  # Raises an error for bad responses
        
        # Parse JSON response
        etf_stock_exposure = response.json()
        
        # Create a DataFrame from the JSON response
        df_etf_stock_exposure = pd.DataFrame(etf_stock_exposure)
        
        # Check if the DataFrame is empty
        if df_etf_stock_exposure.empty:
            print(f"No data available for the provided ETF symbol: {symbol}")
        else:
            # Sort the DataFrame by weightPercentage in descending order
            df_etf_stock_exposure = df_etf_stock_exposure.sort_values(by='weightPercentage', ascending=False)
            
            # Select relevant columns
            df_etf_stock_exposure = df_etf_stock_exposure[['etfSymbol', 'weightPercentage', 'assetExposure']]
            
            # Rename columns for clarity
            df_etf_stock_exposure.rename(columns={
                'etfSymbol': 'ETF', 
                'assetExposure': 'Ticker', 
                'weightPercentage': 'ETF Weightage'
            }, inplace=True)
            
            # # Add a column to indicate the source ETF symbol
            # df_etf_stock_exposure['Source ETF'] = symbol
            
            # Append the DataFrame to the list
            df_list.append(df_etf_stock_exposure)

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred for {symbol}: {http_err}")
    except Exception as err:
        print(f"An error occurred for {symbol}: {err}")

# Concatenate all DataFrames in the list into a single DataFrame
if df_list:
    combined_df = pd.concat(df_list, ignore_index=True)
    display(combined_df)
else:
    print("No data retrieved for any of the specified ETFs.")


In [189]:
combined_df

,ETF,ETF Weightage,Ticker
0,PSL,3.950000,SFM
1,PBJ,3.662000,SFM
2,XMMO,2.834000,SFM
3,OMFS,2.822000,SFM
4,QMOM,2.230000,SFM
...,...,...,...
987,BND,0.000000,AAPL
988,GINN,0.000000,AAPL
989,ERN1.L,0.000000,AAPL
990,EQLS,-0.197644,AAPL


In [187]:

sector_etfs = [
    # 'XLP',  # Consumer Staples
    # 'XLY',  # Consumer Discretionary
    # 'XLI',  # Industrials
    # 'XLB',  # Materials
    # 'XLC',  # Communication Services
    # 'XLF',  # Financials
    # 'XLV',  # Health Care
    # 'XLI',  # Industrials
    # 'XLU',  # Utilities
    # 'XLE',  # Energy
    # 'XRT',  # Retail
    # 'XLK',  # Technology
    # 'XLRE',  # Real Estate
    # 'VOO',  # S&P 500 ETF
    # 'SPY',  # S&P 500 ETF
    # 'QQQ',  # Nasdaq-100 ETF
    # 'IWM',  # Russell 2000 ETF (Small-Cap)
    # 'DIA',  # Dow Jones Industrial Average ETF
    # 'SPYV', # S&P 500 Value ETF
    # 'SPYG', # S&P 500 Growth ETF
    # 'RSP',  # S&P 500 Equal Weight ETF
    'ARKK',
    'PAVE',
    'KBE',
    'XLY',
    'VGT',
    'PDP'
]

# Initialize an empty DataFrame to store all top tickers
final_top_tickers_df = pd.DataFrame()

# Loop through each sector ETF
for sector_etf in sector_etfs:
    # Filter combined_df for the current sector ETF
    sector_df = combined_df[combined_df['ETF'] == sector_etf]
    
    # Sort the filtered DataFrame by 'ETF Weightage' in descending order and get the top 3
    top_tickers = sector_df.nlargest(3, 'ETF Weightage')
    
    # Proceed only if there are top tickers found
    if not top_tickers.empty:
        # Add a new column to indicate the source ETF symbol
        top_tickers['Source ETF'] = sector_etf
        
        # Concatenate to the final DataFrame (axis=0 for row-wise)
        final_top_tickers_df = pd.concat([final_top_tickers_df, top_tickers], ignore_index=True)

# Final DataFrame should now contain information about top tickers for each sector ETF.
if not final_top_tickers_df.empty:
    # Display the final DataFrame
    display(final_top_tickers_df[['ETF', 'Ticker', 'ETF Weightage', 'Source ETF']])
else:
    print("No top tickers found for the provided sector ETFs.")


,ETF,Ticker,ETF Weightage,Source ETF
0,VGT,AAPL,16.090,VGT
1,PDP,AAPL,2.762,PDP


In [ ]:
import requests
import json
import certifi
from urllib.request import urlopen

def get_jsonparsed_data(url):
    try:
        response = urlopen(url, cafile=certifi.where())
        data = response.read().decode("utf-8")
        return json.loads(data)
    except Exception as e:
        print(f"Error fetching data from {url}: {e}")
        return None

api_key = 'Eoqg7H1rEbKrUcPQqj5dU8aKVHd5Mlx4'
symbol = 'AAPL'

# ================================
# 1. Market Capitalization
# ================================
url_market_capitalization = f"https://financialmodelingprep.com/api/v3/market-capitalization/{symbol}?apikey={api_key}"
url_historical_market_capitalization = f"https://financialmodelingprep.com/api/v3/historical-market-capitalization/{symbol}?limit=100&from=2023-10-10&to=2023-12-10&apikey={api_key}"

# ================================
# 2. Key Metrics
# ================================
url_key_metrics_annual = f"https://financialmodelingprep.com/api/v3/key-metrics/{symbol}?period=annual&apikey={api_key}"
url_key_metrics_ttm = f"https://financialmodelingprep.com/api/v3/key-metrics-ttm/{symbol}?apikey={api_key}"
url_ratios_ttm = f"https://financialmodelingprep.com/api/v3/ratios-ttm/{symbol}?apikey={api_key}"

# ================================
# 3. Financial Statement Growth
# ================================
url_cash_flow_growth = f"https://financialmodelingprep.com/api/v3/cash-flow-statement-growth/{symbol}?period=annual&apikey={api_key}"
url_income_statement_growth = f"https://financialmodelingprep.com/api/v3/income-statement-growth/{symbol}?period=annual&apikey={api_key}"
url_balance_sheet_growth = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement-growth/{symbol}?period=annual&apikey={api_key}"
url_financial_growth = f"https://financialmodelingprep.com/api/v3/financial-growth/{symbol}?period=annual&apikey={api_key}"

# ================================
# 4. Valuation Models
# ================================
url_discounted_cash_flow = f"https://financialmodelingprep.com/api/v3/discounted-cash-flow/{symbol}?apikey={api_key}"

# ================================
# 5. Ratings and Surprises
# ================================
url_rating = f"https://financialmodelingprep.com/api/v3/rating/{symbol}?apikey={api_key}"
url_historical_rating = f"https://financialmodelingprep.com/api/v3/historical-rating/{symbol}?apikey={api_key}"
url_earnings_surprises = f"https://financialmodelingprep.com/api/v3/earnings-surprises/{symbol}?apikey={api_key}"

# ================================
# 6. Articles and News
# ================================
url_latest_articles = f"https://financialmodelingprep.com/api/v3/fmp/articles?page=0&size=5&apikey={api_key}"

# ================================
# 7. ETF Information
# ================================
url_etf_holder = f"https://financialmodelingprep.com/api/v3/etf-holder/{symbol}?apikey={api_key}"

url_etf_stock_exposure = f"https://financialmodelingprep.com/api/v3/etf-stock-exposure/{symbol}?apikey={api_key}"

# ================================
# 8. Sector Performance
# ================================
url_sectors_performance = f"https://financialmodelingprep.com/api/v3/sectors-performance?apikey={api_key}"
url_historical_sectors_performance = f"https://financialmodelingprep.com/api/v3/historical-sectors-performance?from=2024-01-01&to=2024-03-01&apikey={api_key}"

# ================================
# 9. Stock Market Data
# ================================
url_stock_market_gainers = f"https://financialmodelingprep.com/api/v3/stock_market/gainers?apikey={api_key}"
url_stock_market_losers = f"https://financialmodelingprep.com/api/v3/stock_market/losers?apikey={api_key}"
url_stock_market_actives = f"https://financialmodelingprep.com/api/v3/stock_market/actives?apikey={api_key}"

# ================================
# Example of Making Requests
# ================================
# Get Market Capitalization
try:
    response_market_cap = requests.get(url_market_capitalization)
    response_market_cap.raise_for_status()  # Raises an error for bad responses
    data_market_cap = response_market_cap.json()  # Parse JSON response

    # Get Historical Market Capitalization
    response_historical_market_cap = requests.get(url_historical_market_capitalization)
    response_historical_market_cap.raise_for_status()  # Raises an error for bad responses
    data_historical_market_cap = response_historical_market_cap.json()

    # Output sample data
    print("Market Capitalization:", data_market_cap)
    print("Historical Market Capitalization:", data_historical_market_cap)

    # Fetch stock market actives
    actives_data = get_jsonparsed_data(url_stock_market_actives)
    print("Stock Market Actives:", actives_data)

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")


Market Capitalization: [{'symbol': 'AAPL', 'date': '2024-11-06', 'marketCap': 3366590976000}]
Historical Market Capitalization: [{'symbol': 'AAPL', 'date': '2023-12-08', 'marketCap': 3035415716730}, {'symbol': 'AAPL', 'date': '2023-12-07', 'marketCap': 3013081658010}, {'symbol': 'AAPL', 'date': '2023-12-06', 'marketCap': 2982837620160}, {'symbol': 'AAPL', 'date': '2023-12-05', 'marketCap': 2999898359460}, {'symbol': 'AAPL', 'date': '2023-12-04', 'marketCap': 2938014405090}, {'symbol': 'AAPL', 'date': '2023-12-01', 'marketCap': 2966087076120}, {'symbol': 'AAPL', 'date': '2023-11-30', 'marketCap': 2946079481850}, {'symbol': 'AAPL', 'date': '2023-11-29', 'marketCap': 2937083819310}, {'symbol': 'AAPL', 'date': '2023-11-28', 'marketCap': 2953058875200}, {'symbol': 'AAPL', 'date': '2023-11-27', 'marketCap': 2943597919770}, {'symbol': 'AAPL', 'date': '2023-11-24', 'marketCap': 2946389677110}, {'symbol': 'AAPL', 'date': '2023-11-22', 'marketCap': 2967172759530}, {'symbol': 'AAPL', 'date': '202

/var/folders/mq/yl1c66r91wb1n1hmpbygjj040000gn/T/ipykernel_12504/1876609567.py:8: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


In [ ]:
import requests
import pandas as pd

api_key = 'Eoqg7H1rEbKrUcPQqj5dU8aKVHd5Mlx4'
symbol = 'AAPL'

# Dictionary to store DataFrames
data_dict = {symbol: {}}

# List of URLs to fetch data from
urls = {

    # ================================
   
    # ================================
    # 7. ETF Information
    # ================================
    "ETF Holder": f"https://financialmodelingprep.com/api/v3/etf-holder/{symbol}?apikey={api_key}",
    "ETF Stock Exposure": f"https://financialmodelingprep.com/api/v3/etf-stock-exposure/{symbol}?apikey={api_key}",
}

# Request data from each URL and store in DataFrames
for description, url in urls.items():
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad responses
        data = response.json()  # Get JSON data
      
        # Convert JSON data to DataFrame
        df = pd.DataFrame(data)

        # Store the DataFrame in the nested dictionary
        data_dict[symbol][description] = df
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching {description}: {e}")
    except Exception as e:
        print(f"An error occurred while processing {description}: {e}")


An error occurred while processing Latest Articles: Mixing dicts with non-Series may lead to ambiguous ordering.
An error occurred while fetching ETF Holder: 403 Client Error: Forbidden for url: https://financialmodelingprep.com/api/v3/etf-holder/AAPL?apikey=Eoqg7H1rEbKrUcPQqj5dU8aKVHd5Mlx4


In [3]:
# display(data_dict)

In [6]:
# The data_dict now contains all data organized in a nested dictionary
# Display each DataFrame for the symbol 'AAPL'
# display(data_dict['AAPL']['Market Capitalization'])
# display(data_dict['AAPL']['Historical Market Capitalization'])
# display(data_dict['AAPL']['Key Metrics Annual'])
# display(data_dict['AAPL']['Key Metrics TTM'])
# display(data_dict['AAPL']['TTM Ratios'])
# display(data_dict['AAPL']['Cash Flow Statement Growth'])
# display(data_dict['AAPL']['Income Statement Growth'])
# display(data_dict['AAPL']['Balance Sheet Statement Growth'])
# display(data_dict['AAPL']['Financial Growth'])
# display(data_dict['AAPL']['Discounted Cash Flow'])
# display(data_dict['AAPL']['Rating'])
# display(data_dict['AAPL']['Historical Rating'])
# display(data_dict['AAPL']['Earnings Surprises'])

# display(data_dict['AAPL']['Latest Articles'])

display(data_dict['AAPL']['ETF Holder'])
display(data_dict['AAPL']['ETF Stock Exposure'])
display(data_dict['AAPL']['Sectors Performance'])
display(data_dict['AAPL']['Historical Sectors Performance'])
display(data_dict['AAPL']['Stock Market Gainers'])
display(data_dict['AAPL']['Stock Market Losers'])
display(data_dict['AAPL']['Stock Market Actives'])


KeyError: 'ETF Holder'

In [ ]:
import finnhub
from datetime import datetime
import pandas as pd

# Initialize Finnhub client with your API key
finnhub_client = finnhub.Client(api_key="csm2g0hr01qgp6njn710csm2g0hr01qgp6njn71g")

# Define the stock symbol as a variable input
symbol = "SFM"  # You can change this to any other symbol, e.g., "AAPL"

# ============================
#      Data Fetching
# ============================

# 1. General News - Fetches general news articles from Finnhub
general_news = finnhub_client.general_news('general', min_id=0)

# 2. Company News - Fetches news specific to the given company symbol from a specified date range
company_news = finnhub_client.company_news(symbol, _from="2000-01-01", to=datetime.now().strftime("%Y-%m-%d"))

# 3. Company Peers - Retrieves a list of peer companies for the given symbol
company_peers = finnhub_client.company_peers(symbol)

# ============================
#      Financial Data
# ============================

# 4. Basic Financials - Gets basic financial data for the company (like revenue, earnings, etc.)
basic_financials = finnhub_client.company_basic_financials(symbol, 'all')

# 5. Reported Financials - Retrieves the financial reports (quarterly) that the company has reported
reported_financials = finnhub_client.financials_reported(symbol=symbol, freq='quarterly')

# 6. Company Earnings - Fetches detailed earnings data for the company, limited to the latest 20 entries
company_earnings = finnhub_client.company_earnings(symbol, limit=20)

# ============================
#      Insiders and Sentiment
# ============================

# 7. Insider Transactions - Fetches data on transactions made by insiders for the company
insider_transactions = finnhub_client.stock_insider_transactions(symbol, '2000-01-01', datetime.now().strftime("%Y-%m-%d"))

# 8. Insider Sentiment - Analyzes sentiments from company insiders (indicates their confidence about the stock)
insider_sentiment = finnhub_client.stock_insider_sentiment(symbol, '2000-01-01', datetime.now().strftime("%Y-%m-%d"))

# ============================
#      Regulatory and Corporate Actions
# ============================

# 9. Filings - Gathers regulatory filings made by the company over a specified period
filings = finnhub_client.filings(symbol=symbol, _from="2000-01-01", to=datetime.now().strftime("%Y-%m-%d"))

# 10. IPO Calendar - Collects information about IPOs within a specified date range
ipo_calendar = finnhub_client.ipo_calendar(_from="2000-01-01", to=datetime.now().strftime("%Y-%m-%d"))

# ============================
#      Analyst and Company Profile
# ============================

# 11. Recommendation Trends - Fetches trends in analysts' recommendations for the specified company
recommendation_trends = finnhub_client.recommendation_trends(symbol)

# 12. Company Profile - Retrieves detailed information about the company
company_profile = finnhub_client.company_profile2(symbol=symbol)

# 13. Earnings Calendar - Gathers upcoming earnings dates for the specified company
earnings_calendar = finnhub_client.earnings_calendar(_from="2000-01-01", to=datetime.now().strftime("%Y-%m-%d"), symbol=symbol, international=False)


In [ ]:
# ============================
#      Data Conversion
# ============================

# Convert general_news to DataFrame
df_general_news = pd.DataFrame(general_news)
if not df_general_news.empty:
    df_general_news['datetime'] = pd.to_datetime(df_general_news['datetime'], unit='s')
    df_general_news['datetime'] = df_general_news['datetime'].dt.strftime('%Y-%m-%d')  # Format datetime
    # print("General News DataFrame:")
    # display(df_general_news)

# Convert company_news to DataFrame
df_company_news = pd.DataFrame(company_news)
if not df_company_news.empty:
    df_company_news['datetime'] = pd.to_datetime(df_company_news['datetime'], unit='s')
    df_company_news['datetime'] = df_company_news['datetime'].dt.strftime('%Y-%m-%d')  # Format datetime
    # print("\nCompany News DataFrame:")
    # display(df_company_news)

# Convert company_peers to DataFrame
df_company_peers = pd.DataFrame(company_peers, columns=[symbol])
# print("\nCompany Peers DataFrame:")
# display(df_company_peers)

# Convert basic_financials to DataFrame
df_basic_financials = pd.DataFrame(basic_financials).T  # Transpose for better viewing
if 'metric' in df_basic_financials.index:
    df_basic_financials = df_basic_financials.loc['metric']
    df_basic_financials = pd.DataFrame(df_basic_financials)
    df_basic_financials = df_basic_financials.T.reset_index()
    df_basic_financials.rename(columns={'index': 'ticker'}, inplace=True)
    df_basic_financials.loc[0, 'ticker'] = symbol
    display(df_basic_financials)
else:
    print("Index 'metric' not found in the DataFrame.")
# print("\nBasic Financials DataFrame:")
# display(df_basic_financials)

# Convert reported_financials to DataFrame
df_reported_financials = pd.DataFrame(reported_financials)
# print("\nReported Financials DataFrame:")
# display(df_reported_financials)

# Convert company_earnings to DataFrame
df_company_earnings = pd.DataFrame(company_earnings)
df_company_earnings.rename(columns={'period': 'earning_report_date', 'actual': 'earning_actual', 'estimate': 'earning_estimate', 'surprise': 'earningSurprise', 'surprisePercent': 'earningSurprisePercent'}, inplace=True)
df_company_earnings = df_company_earnings[['symbol', 'year', 'quarter', 'earning_report_date', 'earning_estimate', 'earning_actual', 'earningSurprise', 'earningSurprisePercent']]
df_company_earnings
# print("\nCompany Earnings DataFrame:")
# display(df_company_earnings)

# Convert insider_transactions to DataFrame
df_insider_transactions = pd.DataFrame(insider_transactions['data'])
#if not df_insider_transactions.empty:
    # print("\nInsider Transactions DataFrame:")
    # display(df_insider_transactions)

# Convert insider_sentiment to DataFrame
df_insider_sentiment = pd.DataFrame(insider_sentiment['data'])
#if not df_insider_sentiment.empty:
    # print("\nInsider Sentiment DataFrame:")
    # display(df_insider_sentiment)

# Convert filings to DataFrame
df_filings = pd.DataFrame(filings)
# print("\nFilings DataFrame:")
# display(df_filings)

# Convert IPO calendar to DataFrame
df_ipo_calendar = pd.DataFrame(ipo_calendar['ipoCalendar'])
#if not df_ipo_calendar.empty:
    # print("\nIPO Calendar DataFrame:")
    # display(df_ipo_calendar)

# Convert recommendation_trends to DataFrame
df_recommendation_trends = pd.DataFrame(recommendation_trends)
df_recommendation_trends.rename(columns={'period': 'reco_date'}, inplace=True)
# print("\nRecommendation Trends DataFrame:")
# display(df_recommendation_trends)

# Convert company_profile to DataFrame (if applicable)
df_company_profile = pd.DataFrame([company_profile])  # Wrap in a list to convert to DataFrame
df_company_profile = df_company_profile[['ticker', 'marketCapitalization', 'finnhubIndustry']]
# print("\nCompany Profile DataFrame:")
# display(df_company_profile)

# Convert earnings_calendar to DataFrame
df_earnings_calendar = pd.DataFrame(earnings_calendar['earningsCalendar'])
df_earnings_calendar.rename(columns={'date': 'earning_report_date'}, inplace=True)
df_earnings_calendar = df_earnings_calendar[['symbol', 'year', 'quarter', 'earning_report_date', 'epsEstimate', 'epsActual', 'revenueEstimate', 'revenueActual']]
#if not df_earnings_calendar.empty:
    # print("\nEarnings Calendar DataFrame:")
    # display(df_earnings_calendar)

In [188]:
# # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
# url = 'https://www.alphavantage.co/query?function=EARNINGS&symbol=IBM&apikey=DHB7WL0KJDY10FG1'
# r = requests.get(url)
# data = r.json()

# print(data)

# import csv
# import requests

# # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
# CSV_URL = 'https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&horizon=3month&apikey=DHB7WL0KJDY10FG1'

# with requests.Session() as s:
#     download = s.get(CSV_URL)
#     decoded_content = download.content.decode('utf-8')
#     cr = csv.reader(decoded_content.splitlines(), delimiter=',')
#     my_list = list(cr)
#     for row in my_list:
#         print(row)

In [8]:
import yfinance as yf

# Define the ticker symbol for the NASDAQ-100 Index
nasdaq_ticker = '^IXIC'  # or use '^IXIC' for NASDAQ Composite

# Download historical stock price data for the NASDAQ index
# Adjust the period and interval as desired
nasdaq_data = yf.download(nasdaq_ticker, period='1mo', interval='1d')  # Last 1 month of data

# Display the stock data
print("NASDAQ Index Data:")
print(nasdaq_data)

# Calculate daily price changes
nasdaq_data['Daily Change'] = nasdaq_data['Close'].diff()

# Display the updated data with daily changes
print("\nNASDAQ Index Data with Daily Changes:")
print(nasdaq_data)


[*********************100%%**********************]  1 of 1 completed

NASDAQ Index Data:
                    Open          High           Low         Close  \
Date                                                                 
2024-10-08  18017.929688  18203.039062  17989.699219  18182.919922   
2024-10-09  18179.220703  18302.050781  18133.019531  18291.619141   
2024-10-10  18200.619141  18333.390625  18154.179688  18282.050781   
2024-10-11  18217.730469  18375.529297  18208.439453  18342.939453   
2024-10-14  18426.660156  18547.919922  18423.599609  18502.689453   
2024-10-15  18515.970703  18564.250000  18252.519531  18315.589844   
2024-10-16  18333.289062  18383.109375  18214.960938  18367.080078   
2024-10-17  18537.210938  18541.460938  18368.789062  18373.609375   
2024-10-18  18466.009766  18524.330078  18452.580078  18489.550781   
2024-10-21  18456.480469  18543.580078  18377.630859  18540.009766   
2024-10-22  18451.859375  18620.710938  18413.470703  18579.769531   
2024-10-23  18502.060547  18509.189453  18146.609375  18276.650391   
2